In [1]:
import requests
import os
import json
import pandas as pd
from pandas import json_normalize

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
%env BEARER_TOKEN= AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis 

env: BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis


In [4]:
def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    return "https://api.twitter.com/2/tweets/sample/stream"

In [5]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


In [6]:
def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))
    return response.json()

In [7]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    #print(json.dumps(response.json()))

In [8]:
def set_rules(headers, delete, bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "dog has:images", "tag": "dog pictures"},
        {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))


In [28]:
def get_stream(headers, set, bearer_token):
    
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at&expansions=author_id&user.fields=created_at", headers=headers, stream=True,
    )
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    tweetDict = []
    #dataFrame = pd.DataFrame(columns = ['created_at','compound'])
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            #print(json.dumps(json_response, indent=4, sort_keys=True)) 
            #print(type(json_response['data']['created_at'])) #class str
            
            created_at = pd.to_datetime(json_response['data']['created_at']) #converting to datetime
            text = json_response['data']['text']
            ps = analyzer.polarity_scores(text)
            #tweetDict.append({'created_at':created_at, 'compound':ps['compound']})
            #tweetDict
            #dataframe = pd.DataFrame.from_dict(json_response['data'], orient="index")
            #dataFrame = json_normalize(json_response['data'])
            #dataFrame = pd.DataFrame(tweetDict)
            print("{}… {}".format(created_at, ps))     
            

In [10]:
import threading

In [11]:
def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)


In [29]:
if __name__ == "__main__":
    main()
    

2021-05-24 19:32:18+00:00… {'neg': 0.0, 'neu': 0.773, 'pos': 0.227, 'compound': 0.8798}
2021-05-24 19:32:19+00:00… {'neg': 0.0, 'neu': 0.712, 'pos': 0.288, 'compound': 0.8172}
2021-05-24 19:32:19+00:00… {'neg': 0.0, 'neu': 0.752, 'pos': 0.248, 'compound': 0.765}
2021-05-24 19:32:19+00:00… {'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'compound': -0.0516}
2021-05-24 19:32:19+00:00… {'neg': 0.247, 'neu': 0.753, 'pos': 0.0, 'compound': -0.5574}
2021-05-24 19:32:19+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:32:19+00:00… {'neg': 0.0, 'neu': 0.851, 'pos': 0.149, 'compound': 0.4215}
2021-05-24 19:32:20+00:00… {'neg': 0.148, 'neu': 0.549, 'pos': 0.303, 'compound': 0.8184}
2021-05-24 19:32:20+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:32:20+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:32:21+00:00… {'neg': 0.0, 'neu': 0.836, 'pos': 0.164, 'compound': 0.6249}
2021-05-24 19:32:21+00:00… {'neg': 0.0, 'neu': 1.0

2021-05-24 19:32:59+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:00+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:00+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:00+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:01+00:00… {'neg': 0.0, 'neu': 0.846, 'pos': 0.154, 'compound': 0.5622}
2021-05-24 19:33:01+00:00… {'neg': 0.199, 'neu': 0.519, 'pos': 0.282, 'compound': 0.8176}
2021-05-24 19:33:03+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:03+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:04+00:00… {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.6597}
2021-05-24 19:33:04+00:00… {'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'compound': 0.0772}
2021-05-24 19:33:04+00:00… {'neg': 0.1, 'neu': 0.618, 'pos': 0.283, 'compound': 0.8616}
2021-05-24 19:33:04+00:00… {'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'co

2021-05-24 19:33:37+00:00… {'neg': 0.0, 'neu': 0.831, 'pos': 0.169, 'compound': 0.6209}
2021-05-24 19:33:37+00:00… {'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'compound': -0.0516}
2021-05-24 19:33:38+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:37+00:00… {'neg': 0.0, 'neu': 0.466, 'pos': 0.534, 'compound': 0.9117}
2021-05-24 19:33:38+00:00… {'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.4898}
2021-05-24 19:33:38+00:00… {'neg': 0.0, 'neu': 0.848, 'pos': 0.152, 'compound': 0.3612}
2021-05-24 19:33:38+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:39+00:00… {'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'compound': 0.3182}
2021-05-24 19:33:39+00:00… {'neg': 0.084, 'neu': 0.835, 'pos': 0.08, 'compound': -0.0323}
2021-05-24 19:33:39+00:00… {'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'compound': 0.4696}
2021-05-24 19:33:39+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:33:40+00:00… {'neg': 0.0, 'neu': 1.0

2021-05-24 19:34:17+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:17+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:17+00:00… {'neg': 0.0, 'neu': 0.599, 'pos': 0.401, 'compound': 0.9081}
2021-05-24 19:34:17+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:17+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:17+00:00… {'neg': 0.17, 'neu': 0.742, 'pos': 0.088, 'compound': -0.5102}
2021-05-24 19:34:18+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:18+00:00… {'neg': 0.148, 'neu': 0.643, 'pos': 0.21, 'compound': 0.3818}
2021-05-24 19:34:18+00:00… {'neg': 0.2, 'neu': 0.8, 'pos': 0.0, 'compound': -0.6124}
2021-05-24 19:34:18+00:00… {'neg': 0.057, 'neu': 0.77, 'pos': 0.173, 'compound': 0.5399}
2021-05-24 19:34:18+00:00… {'neg': 0.051, 'neu': 0.741, 'pos': 0.208, 'compound': 0.7071}
2021-05-24 19:34:18+00:00… {'neg': 0.0, 'neu': 0.913, 'pos': 0.0

2021-05-24 19:34:55+00:00… {'neg': 0.0, 'neu': 0.853, 'pos': 0.147, 'compound': 0.5622}
2021-05-24 19:34:56+00:00… {'neg': 0.155, 'neu': 0.75, 'pos': 0.095, 'compound': -0.296}
2021-05-24 19:34:56+00:00… {'neg': 0.029, 'neu': 0.771, 'pos': 0.2, 'compound': 0.8176}
2021-05-24 19:34:56+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:56+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:56+00:00… {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.6597}
2021-05-24 19:34:56+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:56+00:00… {'neg': 0.148, 'neu': 0.759, 'pos': 0.093, 'compound': -0.3832}
2021-05-24 19:34:57+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:34:58+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:35:00+00:00… {'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'compound': 0.8481}
2021-05-24 19:35:00+00:00… {'neg': 0.0, 'neu': 0.81, 'pos': 0.

2021-05-24 19:35:33+00:00… {'neg': 0.0, 'neu': 0.712, 'pos': 0.288, 'compound': 0.8172}
2021-05-24 19:35:34+00:00… {'neg': 0.231, 'neu': 0.769, 'pos': 0.0, 'compound': -0.8016}
2021-05-24 19:35:34+00:00… {'neg': 0.0, 'neu': 0.873, 'pos': 0.127, 'compound': 0.4939}
2021-05-24 19:35:34+00:00… {'neg': 0.199, 'neu': 0.519, 'pos': 0.282, 'compound': 0.8176}
2021-05-24 19:35:35+00:00… {'neg': 0.0, 'neu': 0.543, 'pos': 0.457, 'compound': 0.9565}
2021-05-24 19:35:35+00:00… {'neg': 0.187, 'neu': 0.813, 'pos': 0.0, 'compound': -0.5994}
2021-05-24 19:35:36+00:00… {'neg': 0.142, 'neu': 0.581, 'pos': 0.277, 'compound': 0.2714}
2021-05-24 19:35:35+00:00… {'neg': 0.0, 'neu': 0.679, 'pos': 0.321, 'compound': 0.9344}
2021-05-24 19:35:36+00:00… {'neg': 0.158, 'neu': 0.688, 'pos': 0.153, 'compound': -0.0387}
2021-05-24 19:35:36+00:00… {'neg': 0.06, 'neu': 0.94, 'pos': 0.0, 'compound': -0.1027}
2021-05-24 19:35:37+00:00… {'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.9231}
2021-05-24 19:35:37+00:0

2021-05-24 19:36:12+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:12+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:13+00:00… {'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'compound': 0.34}
2021-05-24 19:36:13+00:00… {'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.5423}
2021-05-24 19:36:14+00:00… {'neg': 0.326, 'neu': 0.674, 'pos': 0.0, 'compound': -0.7003}
2021-05-24 19:36:15+00:00… {'neg': 0.224, 'neu': 0.776, 'pos': 0.0, 'compound': -0.3818}
2021-05-24 19:36:15+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:16+00:00… {'neg': 0.0, 'neu': 0.671, 'pos': 0.329, 'compound': 0.5319}
2021-05-24 19:36:17+00:00… {'neg': 0.0, 'neu': 0.455, 'pos': 0.545, 'compound': 0.802}
2021-05-24 19:36:17+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:17+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:17+00:00… {'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'com

2021-05-24 19:36:48+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:48+00:00… {'neg': 0.199, 'neu': 0.519, 'pos': 0.282, 'compound': 0.8176}
2021-05-24 19:36:51+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:51+00:00… {'neg': 0.056, 'neu': 0.824, 'pos': 0.12, 'compound': 0.4926}
2021-05-24 19:36:51+00:00… {'neg': 0.095, 'neu': 0.765, 'pos': 0.141, 'compound': 0.3111}
2021-05-24 19:36:51+00:00… {'neg': 0.083, 'neu': 0.917, 'pos': 0.0, 'compound': -0.3291}
2021-05-24 19:36:52+00:00… {'neg': 0.05, 'neu': 0.849, 'pos': 0.101, 'compound': 0.3818}
2021-05-24 19:36:52+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:52+00:00… {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2021-05-24 19:36:52+00:00… {'neg': 0.179, 'neu': 0.597, 'pos': 0.224, 'compound': 0.1531}
2021-05-24 19:36:53+00:00… {'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'compound': 0.7081}
2021-05-24 19:36:52+00:00… {'neg': 0.0, 'neu': 0.84

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))